# 文本预处理

文本是一类序列数据，一篇文章可以看做是字符或者单词的序列。本节将介绍文本数据的常见的预处理步骤。

预处理通常包括四个步骤：

    1.读入文本
    2.分词
    3.建立词典，将每个词映射到一个唯一的索引（index）
    4.将文本从词的序列转换为索引的序列，方便输入模型
    

## 读入文本

我们用一部英文小说，即H. G. Well的[Time Machine](http://www.gutenberg.org/ebooks/35)，作为示例，展示文本预处理的具体过程。

In [32]:
import collections
import re

def read_time_machine():
    with open(r'C:/000Disk/Learn/PyTorch/Dive-into-DL-PyTorch-master/data/thetimemachine.txt', 'r') as f:
        lines = [re.sub('[^a-z]+', ' ', line.strip().lower()) for line in f]
    return lines

lines = read_time_machine()
print('# sentences %d' % len(lines))

# sentences 1000


### bad character range a-Z at position 2
this issue will look into it later

 应该是a-z， 而不是a-Z。连接符- 应该是ASCII码值从小到大，不然的话就会报错

## 分词

我们对每个句子进行分词，也就是将一个句子划分成若干个词（token），转换为一个词的序列。

In [33]:
def tokenize(sentences, token='word'):
    """split sentences into word or char tokens"""
    if token == 'word':
        return [sentence.split(' ') for sentence in sentences]
    elif token == 'char':
        return [list(sentence) for sentence in sentences]
    else:
        print('ERROR: unknow toke type' + token)

tokens = tokenize(lines)
tokens[0:2]

[['the', 'time', 'machine', 'by', 'h', 'g', 'wells'], ['']]

## 建立字典

为了方便模型处理，我们需要将字符串转换为数字。因此我们需要先构建一个字典（vocabulary），将每个词映射到一个唯一的索引编号。

In [34]:
class Vocab(object): 
    def __init__(self, tokens, min_freq=0, use_special_tokens=False):
        counter = count_corpus(tokens)
        self.token_freqs = list(counter.items())
        self.idx_to_token = []
        if use_special_tokens:
            self.pad, self.bos, self.eos, self.unk = (0,1,2,3)
        else:
            self.unk = 0
            self.idx_to_token += ['']
        self.idx_to_token += [token for token, freq in self.token_freqs
                             if freq >= min_freq and token not in self.idx_to_token]
        self.token_to_idx = dict()
        for idx, token in enumerate(self.idx_to_token):
            self.token_to_idx[token] = idx
            
    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self, tokens):
        if not isinstance(tokens,(list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]
    
    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]


def count_corpus(sentences):
    tokens = [tk for st in sentences for tk in st]
    return collections.Counter(tokens)

我们看一个例子，这里我们尝试用Time Machine作为语料构建字典

In [35]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[0:10])

[('', 0), ('the', 1), ('time', 2), ('machine', 3), ('by', 4), ('h', 5), ('g', 6), ('wells', 7), ('i', 8), ('traveller', 9)]


# 将词转为索引

使用字典，我们可以将原本文中的句子从单词序列转换为索引序列

In [36]:
for i in range(8, 10):
    print('words:', tokens[i])
    print('indices:', vocab[tokens[i]])

words: ['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
indices: [1, 2, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
words: ['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
indices: [20, 21, 22, 23, 24, 16, 25, 26, 27, 28, 29, 30]


## 用现有工具进行分词

我们前面介绍的分词方式非常简单，它至少有以下几个缺点:

1. 标点符号通常可以提供语义信息，但是我们的方法直接将其丢弃了
2. 类似“shouldn't", "doesn't"这样的词会被错误地处理
3. 类似"Mr.", "Dr."这样的词会被错误地处理

我们可以通过引入更复杂的规则来解决这些问题，但是事实上，有一些现有的工具可以很好地进行分词，我们在这里简单介绍其中的两个：[spaCy](https://spacy.io/)和[NLTK](https://www.nltk.org/)。

下面是一个简单的例子：

In [37]:
text = "Mr. Chen doesn't agree with my suggestion."

In [ ]:
!python -m spacy download en

## spaCy:

#en库安装
python -m spacy download en

#en_core_web_md库安装
python -m spacy download en_core_web_md

#en_core_web_sm库安装
python -m spacy download en_core_web_sm

# 链接安装
pip install /Users/you/en_core_web_md-1.2.0.tar.gz
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-1.2.0/en_core_web_md-1.2.0.tar.gz 

In [43]:
import spacy
nlp = spacy.load('en')
doc = nlp(text)
print([token.text for token in doc])

OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

# NLTK

In [ ]:
import nltk

## Stanford NLP

from nltk.tag.stanford import StanfordNERTagger

# 中文分词
## pip3 install jieba

# 习题

选择题

### 1.下列哪一项不是构建Vocab类所必须的步骤：

A.词频统计，清洗低频词

B.句子长度统计

C.构建索引到token的映射

D.构建token到索引的映射

答案： B

答案解释: 句子长度统计与构建字典无关

### 2.无论use_special_token参数是否为真，都会使用的特殊token是____，作用是用来____。

A.$<pad>$，在小批量中数据不等长时做填充

B.$<bos>$，标记句子的开始

C.$<eos>$，标记句子的结束

D.$<unk>$，表示未登录词

答案：D